In [1]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_log_error, mean_absolute_error
import numpy as np

In [2]:
def loss_calc(position, combined=False):

    #load in fantasypros preseason rankings for one position
    position = position.upper()
    lower_position = position.lower()
    fan_proj = pd.read_csv("data/espn_and_nflcom_preseason_projs/FantasyPros_Fantasy_Football_Projections_"+position+".csv")

    #add in a column at beginning with preseason rank by using order of rows
    num_players = len(fan_proj)
    ranks = np.arange(1, num_players+1, 1)
    fan_proj.insert(0, "Experts_Rank", ranks)
    fan_proj.rename(columns={"NAME": "Player", "FPTS": "Experts_Projections"}, inplace=True)

    my_proj = None

    #load in projections from best model (check github for these)
    if combined == True:
        my_proj = pd.read_csv('draft_proj_083022/'+position+'/'+lower_position+'_combined_projs.csv')
    else:
        if position == "QB":
            my_proj= pd.read_csv('projections/QB/BayesianRidge2022_projections_20220830-142824.csv')
        elif position == "RB":
            my_proj= pd.read_csv('projections/RB/RandomForestRegressor2022_projections_20220830-142827.csv')
        elif position == "WR":
            my_proj= pd.read_csv('projections/WR/BayesianRidge2022_projections_20220830-142827.csv')
        else:
            my_proj= pd.read_csv('projections/TE/LinearRegression2022_projections_20220830-142828.csv')
    num_players = len(my_proj)
    ranks = np.arange(1, num_players+1, 1)
    my_proj.insert(0, "My_Rank", ranks)

    #load in fantasy actual results
    player_actuals = pd.read_csv("data/fantasy_actual_results/"+position+"_fantasy_actuals_2022.csv")
    num_players = len(player_actuals)
    ranks = np.arange(1, num_players+1, 1)
    player_actuals.insert(0, "Actual_Rank", ranks)
    player_actuals.rename(columns={"NAME": "Player", "FPTS": "Actual_FPTS"}, inplace=True)


    #create table to store loss statistics
    loss_stats = []

    #create df for desired columns
    results_df = pd.DataFrame()
    results_df["Player"] = player_actuals['Player']
    results_df["Actual_FPTS"] = player_actuals['Actual_FPTS']
    results_df["Actual_Rank"] = player_actuals['Actual_Rank']
    results_df = results_df.merge(fan_proj[['Player', 'Experts_Projections', 'Experts_Rank']], how='inner', on='Player')
    results_df = results_df.merge(my_proj[['Player', 'Model_Projection_Points', 'My_Rank']], how='inner', on='Player')
    results_df = results_df[['Player','Actual_FPTS','Experts_Projections','Model_Projection_Points','Actual_Rank','Experts_Rank','My_Rank']]


    #calculate loss metrics against Fantasy Football Rankings
    experts_points_mae = mean_absolute_error(results_df['Experts_Projections'],results_df['Actual_FPTS'])
    print("Expert Points MAE: "+ str(experts_points_mae))
    my_points_mae = mean_absolute_error(results_df['Model_Projection_Points'],results_df['Actual_FPTS'])
    print("My Points MAE: "+ str(my_points_mae))
    experts_ranks_mae = mean_absolute_error(results_df['Experts_Rank'],results_df['Actual_Rank'])
    print("Expert Rank MAE: "+ str(experts_ranks_mae))
    my_ranks_mae = mean_absolute_error(results_df['My_Rank'],results_df['Actual_Rank'])
    print("My Rank MAE: "+ str(my_ranks_mae))

    model_maes = [experts_points_mae, my_points_mae, experts_ranks_mae, my_ranks_mae]

    return model_maes, results_df


In [8]:
maes, results = loss_calc('QB')

Expert Points MAE: 78.74266666666666
My Points MAE: 76.9332775848405
Expert Rank MAE: 7.433333333333334
My Rank MAE: 9.366666666666667


In [10]:
maes, results = loss_calc('RB')

Expert Points MAE: 51.25333333333334
My Points MAE: 58.919413860830495
Expert Rank MAE: 14.277777777777779
My Rank MAE: 14.537037037037036


In [12]:
maes, results = loss_calc('WR')

Expert Points MAE: 59.51163636363637
My Points MAE: 47.342645119904425
Expert Rank MAE: 17.581818181818182
My Rank MAE: 17.78181818181818


In [14]:
maes, results = loss_calc('TE')

Expert Points MAE: 46.76538461538462
My Points MAE: 35.546342881942934
Expert Rank MAE: 11.73076923076923
My Rank MAE: 12.346153846153847
